In [1]:
import os
import argparse
import math
from decimal import Decimal
from os.path import join

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import pandas as pd
from tqdm import tqdm

from tractseg.data import dataset_specific_utils
from tractseg.libs.AFQ_MultiCompCorrection import AFQ_MultiCompCorrection
from tractseg.libs.AFQ_MultiCompCorrection import get_significant_areas
from tractseg.libs import metric_utils
from tractseg.libs import plot_utils
from tractseg.libs import tracking
import glob as glob
import nibabel as nib
from scipy.stats import t as t_dist

In [2]:
def parse_subjects_file(file_path):
    with open(file_path) as f:
        l = f.readline().strip()
        if l.startswith("# tractometry_path="):
            base_path = l.split("=")[1]
        else:
            raise ValueError("Invalid first line in subjects file. Must start with '# tractometry_path='")

        bundles = None
        plot_3D_path = None

        # parse bundle names
        for i in range(2):
            l = f.readline().strip()
            if l.startswith("# bundles="):
                bundles_string = l.split("=")[1]
                bundles = bundles_string.split(" ")

                valid_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
                for bundle in bundles:
                    if bundle not in valid_bundles:
                        raise ValueError("Invalid bundle name: {}".format(bundle))

                print("Using {} manually specified bundles.".format(len(bundles)))
            elif l.startswith("# plot_3D="):
                plot_3D_path = l.split("=")[1]

        #if bundles is None:
        #    bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
    bundles = dataset_specific_utils.get_bundle_names("All")[1:]
    
    df = pd.read_csv(file_path, sep=" ", comment="#")
    df["subject_id"] = df["subject_id"].astype(str)

    # Check that each column (except for first one) is correctly parsed as a number
    for col in df.columns[1:]:
        if not np.issubdtype(df[col].dtype, np.number):
            raise IOError("Column {} contains non-numeric values".format(col))

    #if df.columns[1] == "group":
    #    if df["group"].max() > 1:
    #        raise IOError("Column 'group' may only contain 0 and 1.")

    return base_path, df, bundles, plot_3D_path

In [3]:
def correct_for_confounds(values, meta_data, bundles, selected_bun_indices, NR_POINTS, analysis_type, confound_names):
    values_cor = np.zeros([len(bundles), NR_POINTS, len(meta_data)])
    for b_idx in selected_bun_indices:
        for jdx in range(NR_POINTS):
            target = np.array([values[s][b_idx][jdx] for s in meta_data["subject_id"]])
            if analysis_type == "group":
                target_cor = metric_utils.unconfound(target, meta_data[["group"] + confound_names].values,
                                                     group_data=True)
            else:
                target_cor = metric_utils.unconfound(target, meta_data[confound_names].values,
                                                     group_data=False)
                meta_data["target"] = metric_utils.unconfound(meta_data["target"].values,
                                                              meta_data[confound_names].values,
                                                              group_data=False)
            values_cor[b_idx, jdx, :] = target_cor

    # Restore original data structure
    values_cor = values_cor.transpose(2, 0, 1)
    # todo: nicer way: use numpy array right from beginning instead of dict
    values_cor_dict = {}
    for idx, subject in enumerate(list(meta_data["subject_id"])):
        values_cor_dict[subject] = values_cor[idx]
    return values_cor_dict

In [4]:
def get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B, alpha, bundles, nperm, b_idx):
    if analysis_type == "group":
        y = np.array((0,) * len(subjects_A) + (1,) * len(subjects_B))
    else:
        y = meta_data["target"].values
    alphaFWE, statFWE, clusterFWE, stats = AFQ_MultiCompCorrection(np.array(values_allp), y,
                                                                   alpha, nperm=nperm)                                                              
    #print("Processing {}...".format(bundles[b_idx]))
    #print("  cluster size: {}".format(clusterFWE))
    #print("  alphaFWE: {}".format(format_number(alphaFWE)))
    #return alphaFWE, clusterFWE,stats
    return alphaFWE, clusterFWE

In [5]:
def format_number(num):
    if abs(num) > 0.00001:
        return round(num, 4)
    else:
        return '%.2e' % Decimal(num)

In [6]:
#FWE_method = "alphaFWE"
FWE_method = "clusterFWE"
show_detailed_p = False
hide_legend = False
show_color_bar = True  # colorbar on 3D plot
nperm = 5000
alpha=0.05
correct_mult_tract_comp = False
base_path, meta_data, selected_bundles, plot_3D_path = parse_subjects_file("/mnt/d//LINUX/CogPhenoPark/dataTractSeg/Tractometry_FA.txt")
analysis_type = "group"

plot_3D_type="pval"
tracking_format="tck"
tracking_dir="FOD_iFOD2_trackings"
output_path="/mnt/d//LINUX/CogPhenoPark" 

Using 31 manually specified bundles.


In [7]:
all_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
values = {}
for subject in meta_data["subject_id"]:
    raw = np.loadtxt(base_path.replace("SUBJECT_ID", subject), delimiter=";", skiprows=1).transpose()
    values[subject] = raw

NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
#selected_bun_indices = [bundles.index(b) for b in selected_bundles]
selected_bun_indices = [all_bundles.index(b) for b in selected_bundles]
print(selected_bun_indices)

OSError: /mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/PARA__sub_640524MB240413.csv not found.

# Two T_Test

In [166]:
correct_mult_tract_comp=True
for withCofound in [True]:
    for para in (['''RD''','''FA''','''MD''','''AD''','''density''']): 
    #for para in ['''RD''']: 
        print("*****************  "+ para+" *********************")
        for group_vs in (['''G1VsG3''','''G3VsG4''','''G2VsG3''','''G1VsG2''','''G1VsG4''','''G2VsG4''']):
        #for group_vs in ['''G1VsG3''']:
            print("*****************  "+ group_vs +" *********************")
            #for ind in [True,False]: 
            for ind in [False]: 
                for show_detailed_p in [True,False]:
                    ###############
                    FWE_method = "alphaFWE"
                    show_detailed_p = False
                    hide_legend = False
                    show_color_bar = True  # colorbar on 3D plot
                    nperm = 5000
                    alpha=0.05
                    correct_mult_tract_comp = False
                    base_path, meta_data, selected_bundles, plot_3D_path = parse_subjects_file("/mnt/d//LINUX/CogPhenoPark/dataTractSeg/Tractometry_template_"+group_vs+".txt")
                    analysis_type = "group"
                    plot_3D_type="none"

                    ###########
                    all_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
                    values = {}
                    for subject in meta_data["subject_id"]:
                        raw = np.loadtxt(base_path.replace("SUBJECT_ID", subject).replace("PARA",para), delimiter=";", skiprows=1).transpose()
                        values[subject] = raw

                    NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
                    selected_bun_indices = [all_bundles.index(b) for b in selected_bundles]

                    confound_names = list(meta_data.columns[2:])

                    cols = 3
                    rows = math.ceil(len(selected_bundles) / cols)

                    a4_dims = (cols*3, rows*5)
                    f, axes = plt.subplots(rows, cols, figsize=a4_dims)

                    axes = axes.flatten()
                    sns.set(font_scale=1.2)
                    sns.set_style("whitegrid")

                    subjects_A = list(meta_data[meta_data["group"] == 0]["subject_id"])
                    subjects_B = list(meta_data[meta_data["group"] == 1]["subject_id"])

                    # Correct for confounds        
                    if withCofound :
                        values = correct_for_confounds(values, meta_data, all_bundles, selected_bun_indices, NR_POINTS, analysis_type,confound_names)

                    # Significance testing with multiple correction of bundles
                    if correct_mult_tract_comp:
                        values_allp = []  # [subjects, NR_POINTS * nr_bundles]
                        for s in meta_data["subject_id"]:
                            print(s)
                            values_subject = []
                            for i, b_idx in enumerate(selected_bun_indices):
            #                    print(b_idx)
            #                    print(np.mean(values[s][b_idx]))
                                values_subject += list(values[s][b_idx]) # concatenate all bundles
                            values_allp.append(values_subject)
                        alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B, alpha,all_bundles, nperm, b_idx)

                    for i, b_idx in enumerate(selected_bun_indices):
                        ############
                        vals_thresA=np.zeros([])
                        subjects_AA=subjects_A[:]
                        for subject in subjects_A:
                            if np.all(values[subject][b_idx]>0) :
                                vals_thresA=np.append(vals_thresA,np.mean(values[subject][b_idx]))
                        vals_thresA=vals_thresA[1:]
                        vals_thresA = vals_thresA[~ np.isnan(vals_thresA)]
                        val_thresA=np.mean(vals_thresA)-2*np.std(vals_thresA)
                        if val_thresA < 0 : val_thresA = 0
                        #print("valeur seuil G0= "+str(val_thresA))

                        vals_thresB=np.zeros([])
                        subjects_BB=subjects_B[:]
                        for j, subject in enumerate(subjects_B):
                            if np.all(values[subject][b_idx]>0) :
                                vals_thresB=np.append(vals_thresB,np.mean(values[subject][b_idx]))
                        vals_thresB=vals_thresB[1:]
                        vals_thresB = vals_thresB[~ np.isnan(vals_thresB)]
                        val_thresB=np.mean(vals_thresB)-2*np.std(vals_thresB)
                        if val_thresB < 0 : val_thresB = 0
                        #print("valeur seuil G1= "+str(val_thresB))

                        # Bring data into right format for seaborn
                        data = {"position": [],
                                "fa": [],
                                "group": [],
                                "subject": []}
                        subjects_AA=subjects_A[:]
                        for j, subject in enumerate(subjects_A):
                            if ((np.mean(values[subject][b_idx]) > val_thresA) & (np.all(values[subject][b_idx]>0))) :
                                for position in range(NR_POINTS):
                                    data["position"].append(position)
                                    data["subject"].append(subject)
                                    data["fa"].append(values[subject][b_idx][position])
                                    data["group"].append(group_vs[0:2])                        
                            else :
                                #print(group_vs[0:2] + " : "+subject+" "+str(np.mean(values[subject][b_idx])))    
                                subjects_AA.remove(subject)
                                
                        subjects_BB=subjects_B[:]
                        for j, subject in enumerate(subjects_B):
                            if ((np.mean(values[subject][b_idx]) > val_thresB) & (np.all(values[subject][b_idx]>0))) :
                                for position in range(NR_POINTS):
                                    data["position"].append(position)
                                    data["subject"].append(subject)
                                    data["fa"].append(values[subject][b_idx][position])
                                    data["group"].append(group_vs[-2:])                        
                            else :
                                #print(group_vs[-2:]+ " : "+subject+" "+str(np.mean(values[subject][b_idx])))                    
                                subjects_BB.remove(subject)
                                
                        # Plot
                        if ind :
                            ax = sns.lineplot(x="position", y="fa", data=data,markers=True,ax=axes[i], hue="group",units="subject",estimator=None, lw=1)  # each subject as single line
                        else :
                            ax = sns.lineplot(x="position", y="fa", data=data,markers=True,ax=axes[i], hue="group")

                        ax.set(xlabel='position along tract', ylabel=para)
                        ax.set_title(all_bundles[b_idx])
                        if analysis_type == "correlation" or hide_legend:
                            ax.legend_.remove()
                        elif analysis_type == "group" and i > 0:
                            ax.legend_.remove()  # only show legend on first subplot

                        alpha=0.05
                        nperm=1000
                        # Significance testing without multiple correction of bundles
                        if not correct_mult_tract_comp:
                            values_allp = [values[s][b_idx] for s in subjects_A + subjects_B]  # [subjects, NR_POINTS]
                            #alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B,alpha, bundles, nperm, b_idx)
                            alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_A, subjects_B,alpha,all_bundles, nperm, b_idx)

                        # Calc p-values
                        pvalues = np.zeros(NR_POINTS)
                        stats = np.zeros(NR_POINTS)  # for ttest: t-value, for pearson: correlation
                        for jdx in range(NR_POINTS):
                            if analysis_type == "group":
                                values_controls = [values[s][b_idx][jdx] for s in subjects_AA]
                                values_patients = [values[s][b_idx][jdx] for s in subjects_BB]
                                stats[jdx], pvalues[jdx] = scipy.stats.ttest_ind(values_controls, values_patients)
                            else:
                                values_controls = [values[s][b_idx][jdx] for s in subjects_A]
                                stats[jdx], pvalues[jdx] = scipy.stats.pearsonr(values_controls, meta_data["target"].values)


                        # Plot significant areas
                        if show_detailed_p:
                            ax2 = axes[i].twinx()
                            ax2.bar(range(len(pvalues)), -np.log10(pvalues), color="gray", edgecolor="none", alpha=0.5)
                            ax2.plot([0, NR_POINTS-1], (-np.log10(alphaFWE),)*2, color="red", linestyle=":")
                            ax2.set(xlabel='position', ylabel='-log10(p)')
                        else:
                            sig_areas = get_significant_areas(pvalues, 1, alphaFWE)
                            sig_areas = sig_areas * np.quantile(np.array(data["fa"]), 0.98)
                            sig_areas[sig_areas == 0] = np.quantile(np.array(data["fa"]), 0.02)
                            axes[i].plot(range(len(sig_areas)), sig_areas, color="red", linestyle=":")
                            sig_areas2 = get_significant_areas(pvalues, clusterFWE, alpha)
                            sig_areas2 = sig_areas2 * np.quantile(np.array(data["fa"]), 0.98)
                            sig_areas2[sig_areas2 == 0] = np.quantile(np.array(data["fa"]), 0.02)
                            axes[i].plot(range(len(sig_areas2)), sig_areas2, color="green", linestyle="-")

                            
                        if np.any(pvalues<alphaFWE):
                            #print(pvalues)
                            print(all_bundles[b_idx])
                            print(para)
                            print(len(subjects_A)+len(subjects_B))
                            print(len(subjects_A))
                            print(len(subjects_B))                            
                            print(len(subjects_AA)+len(subjects_BB))
                            print(len(subjects_AA))
                            print(len(subjects_BB))

                        # Plot text
                        axes[i].annotate("alphaFWE:   {}".format(format_number(alphaFWE)),
                                         (0, 0), (0, -45), xycoords='axes fraction', textcoords='offset points', va='top',
                                         fontsize=10)
                        axes[i].annotate("min p-value: {}".format(format_number(pvalues.min())),
                                         (0, 0), (0, -65), xycoords='axes fraction', textcoords='offset points', va='top',
                                         fontsize=10)
                        axes[i].annotate("clusterFWE:   {}".format(clusterFWE),
                                         (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
                                         fontsize=10)
                        STR="n : "+str(len(subjects_AA)+len(subjects_BB))+"/"+str(len(subjects_AA))+"/"+str(len(subjects_BB))
                        axes[i].annotate(STR,
                                         (0, 0), (0, -75), xycoords='axes fraction', textcoords='offset points', va='top',
                                         fontsize=10)
                        
                        stats_label = "t-value:      " if analysis_type == "group" else "corr.coeff.: "
                        #axes[i].annotate(stats_label + "   {}".format(format_number(stats[pvalues.argmin()])),
                        #                 (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
                        #                 fontsize=10)

                        if plot_3D_type != "none":
                            print(plot_3D_type)
                            if plot_3D_type == "metric":
                                metric = np.array([values[s][b_idx] for s in subjects_A + subjects_B]).mean(axis=0)
                            else:
                                #metric = pvalues  # use this code if you want to plot the pvalues instead of the FA
                                metric = sig_areas

                            #bundle = bundles[b_idx]
                            bundle = all_bundles[b_idx]
                            output_path_3D = output_path.split(".")[0] +"/"+bundle+"_"+para+"_"+group_vs+"_2std_c._3D.png"

                            if tracking_dir == "auto":
                                tracking_dir = tracking.get_tracking_folder_name("fixed_prob", False)

                            if tracking_format == "tck":
                                tracking_path = join(plot_3D_path, tracking_dir, bundle + ".tck")
                            else:
                                tracking_path = join(plot_3D_path, tracking_dir, bundle + ".trk")
                            ending_path = join(plot_3D_path, "endings_segmentations", bundle + "_b.nii.gz")
                            mask_path = join(plot_3D_path, "nodif_brain_mask.nii.gz")

                            if not os.path.isfile(tracking_path):
                                raise ValueError("Could not find: " + tracking_path)
                            if not os.path.isfile(ending_path):
                                raise ValueError("Could not find: " + ending_path)
                            if not os.path.isfile(mask_path):
                                raise ValueError("Could not find: " + mask_path)

                            print(tracking_path)
                            print(ending_path)
                            print(mask_path)
                            print(bundle)
                            print(metric)
                            print(output_path_3D)
                            print(tracking_format)
                            print(show_color_bar)
                            plot_utils.plot_bundles_with_metric(tracking_path, ending_path, mask_path, bundle, metric,
                                                                output_path_3D, tracking_format, show_color_bar)


                    plt.tight_layout()
                    plt.savefig("/mnt/d//LINUX/CogPhenoPark/"+para+"_"+group_vs+"_ind_"+str(ind)+'_p_'+str(show_detailed_p)+"_cofound_"+str(withCofound)+"_2std_cc_mtc.png", dpi=200)
                    plt.close ('all')

*****************  RD *********************
*****************  G1VsG3 *********************
Using 19 manually specified bundles.
CC_7
RD
64
41
23
24
11
13
ILF_right
RD
64
41
23
44
28
16
Using 19 manually specified bundles.
CC_7
RD
64
41
23
24
11
13
ILF_right
RD
64
41
23
44
28
16
*****************  G3VsG4 *********************
Using 19 manually specified bundles.
T_OCC_right
RD
53
23
30
52
23
29
UF_left
RD
53
23
30
48
20
28
Using 19 manually specified bundles.
IFO_right
RD
53
23
30
44
19
25
T_OCC_right
RD
53
23
30
52
23
29
UF_left
RD
53
23
30
48
20
28
*****************  G2VsG3 *********************
Using 19 manually specified bundles.
CC_4
RD
38
15
23
36
15
21
CC_6
RD
38
15
23
38
15
23
CC_7
RD
38
15
23
36
14
22
SLF_II_left
RD
38
15
23
37
14
23
Using 19 manually specified bundles.
CC_4
RD
38
15
23
36
15
21
CC_6
RD
38
15
23
38
15
23
CC_7
RD
38
15
23
36
14
22
SLF_II_left
RD
38
15
23
37
14
23
*****************  G1VsG2 *********************
Using 19 manually specified bundles.
STR_left
RD
56

ATR_left
AD
53
23
30
53
23
30
ATR_right
AD
53
23
30
52
23
29
T_OCC_right
AD
53
23
30
52
23
29
UF_left
AD
53
23
30
50
21
29
Using 19 manually specified bundles.
CC_6
AD
53
23
30
52
23
29
IFO_right
AD
53
23
30
50
21
29
ILF_left
AD
53
23
30
53
23
30
SLF_III_left
AD
53
23
30
53
23
30
ATR_left
AD
53
23
30
53
23
30
ATR_right
AD
53
23
30
52
23
29
T_OCC_right
AD
53
23
30
52
23
29
UF_left
AD
53
23
30
50
21
29
*****************  G2VsG3 *********************
Using 19 manually specified bundles.
CC_4
AD
38
15
23
37
15
22
UF_left
AD
38
15
23
36
15
21
Using 19 manually specified bundles.
CC_4
AD
38
15
23
37
15
22
ILF_right
AD
38
15
23
35
15
20
UF_left
AD
38
15
23
36
15
21
*****************  G1VsG2 *********************
Using 19 manually specified bundles.
Using 19 manually specified bundles.
*****************  G1VsG4 *********************
Using 19 manually specified bundles.
CC_4
AD
70
41
29
66
38
28
IFO_right
AD
70
41
29
67
40
27
ILF_right
AD
70
41
29
67
40
27
ATR_right
AD
70
41
29
68
40
28
STR_lef

<ipython-input-166-0bdf25206c37>:170: RuntimeWarning: invalid value encountered in less
  if np.any(pvalues<alphaFWE):


UF_right
density
38
15
23
23
10
13
Using 19 manually specified bundles.
CC_4
density
38
15
23
35
14
21
CC_7
density
38
15
23
38
15
23
SLF_I_right
density
38
15
23
37
14
23
SLF_II_left
density
38
15
23
37
14
23
SLF_III_right
density
38
15
23
37
15
22
STR_right
density
38
15
23
36
14
22


<ipython-input-166-0bdf25206c37>:170: RuntimeWarning: invalid value encountered in less
  if np.any(pvalues<alphaFWE):


UF_right
density
38
15
23
23
10
13
*****************  G1VsG2 *********************
Using 19 manually specified bundles.
ILF_right
density
56
41
15
49
36
13
Using 19 manually specified bundles.
ILF_right
density
56
41
15
49
36
13
*****************  G1VsG4 *********************
Using 19 manually specified bundles.
AF_left
density
70
41
29
67
39
28
CC_4
density
70
41
29
65
39
26
CC_6
density
70
41
29
68
40
28
CC_7
density
70
41
29
68
39
29
IFO_right
density
70
41
29
68
41
27
ILF_left
density
70
41
29
67
39
28
ILF_right
density
70
41
29
67
40
27
SLF_III_left
density
70
41
29
68
40
28
ATR_left
density
70
41
29
69
40
29
STR_right
density
70
41
29
66
39
27
T_OCC_left
density
70
41
29
68
40
28
T_OCC_right
density
70
41
29
67
40
27
UF_right
density
70
41
29
68
40
28
Using 19 manually specified bundles.
CC_4
density
70
41
29
65
39
26
CC_6
density
70
41
29
68
40
28
CC_7
density
70
41
29
68
39
29
IFO_right
density
70
41
29
68
41
27
ILF_left
density
70
41
29
67
39
28
ILF_right
density
70
41
29
67
4

In [ ]:
plot_3D_type="pval"
tracking_format="tck"
tracking_dir="FOD_iFOD2_trackings"
output_path="/mnt/d//LINUX/CogPhenoPark"
print(plot_3D_path)

if plot_3D_type != "none":
                if plot_3D_type == "metric":
                    metric = np.array([values[s][b_idx] for s in subjects_A + subjects_B]).mean(axis=0)
                else:
                    # metric = pvalues  # use this code if you want to plot the pvalues instead of the FA
                    metric = sig_areas

                #bundle = bundles[b_idx]
                bundle = all_bundles[b_idx]
                output_path_3D = output_path.split(".")[0] + "_" + bundle + "_3D.png"

                if tracking_dir == "auto":
                    tracking_dir = tracking.get_tracking_folder_name("fixed_prob", False)

                if tracking_format == "tck":
                    tracking_path = join(plot_3D_path, tracking_dir, bundle + ".tck")
                else:
                    tracking_path = join(plot_3D_path, tracking_dir, bundle + ".trk")
                ending_path = join(plot_3D_path, "endings_segmentations", bundle + "_b.nii.gz")
                mask_path = join(plot_3D_path, "nodif_brain_mask.nii.gz")

                if not os.path.isfile(tracking_path):
                    raise ValueError("Could not find: " + tracking_path)
                if not os.path.isfile(ending_path):
                    raise ValueError("Could not find: " + ending_path)
                if not os.path.isfile(mask_path):
                    raise ValueError("Could not find: " + mask_path)

                plot_utils.plot_bundles_with_metric(tracking_path, ending_path, mask_path, bundle, metric,
                                                    output_path_3D, tracking_format, show_color_bar)



/mnt/d/LINUX/CogPhenoPark/dataTractSeg/T84030


In [133]:
output_path_3D

NameError: name 'output_path_3D' is not defined

# ANOVA

In [162]:
for para in (['''FA''','''RD''','''MD''','''AD''','''density''']): 
    for group_vs in ([''' ''']):

        ###############
        FWE_method = "alphaFWE"
        show_detailed_p = False
        hide_legend = False
        show_color_bar = True  # colorbar on 3D plot
        nperm = 5000
        alpha=0.05
        correct_mult_tract_comp = False
        base_path, meta_data, selected_bundles, plot_3D_path = parse_subjects_file("/mnt/d//LINUX/CogPhenoPark/dataTractSeg/Tractometry_template.txt")
        analysis_type = "group"
        plot_3D_type="none"

        ###########
        all_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
        values = {}
        for subject in meta_data["subject_id"]:
            raw = np.loadtxt(base_path.replace("SUBJECT_ID", subject).replace("PARA",para), delimiter=";", skiprows=1).transpose()
            values[subject] = raw

        NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
        #selected_bun_indices = [bundles.index(b) for b in selected_bundles]
        selected_bun_indices = [all_bundles.index(b) for b in selected_bundles]
        ############

        confound_names = list(meta_data.columns[2:])

        cols = 3
        rows = math.ceil(len(selected_bundles) / cols)

        a4_dims = (cols*3, rows*5)
        f, axes = plt.subplots(rows, cols, figsize=a4_dims)

        axes = axes.flatten()
        sns.set(font_scale=1.2)
        sns.set_style("whitegrid")

        subjects_A = list(meta_data[meta_data["group"] == 0]["subject_id"])
        subjects_B = list(meta_data[meta_data["group"] == 1]["subject_id"])
        subjects_C = list(meta_data[meta_data["group"] == 2]["subject_id"])
        subjects_D = list(meta_data[meta_data["group"] == 3]["subject_id"])
        
        # Correct for confounds
        values = correct_for_confounds(values, meta_data, all_bundles, selected_bun_indices, NR_POINTS, analysis_type,confound_names)

        for i, b_idx in enumerate(tqdm(selected_bun_indices)):
#            print(all_bundles[b_idx])
            ############
            vals_thresA=np.zeros([])
            subjects_AA=subjects_A[:]
            for subject in subjects_A:
                if np.all(values[subject][b_idx]>0) :
                    vals_thresA=np.append(vals_thresA,np.mean(values[subject][b_idx]))
            vals_thresA=vals_thresA[1:]
            vals_thresA = vals_thresA[~ np.isnan(vals_thresA)]
            val_thresA=np.mean(vals_thresA)-2*np.std(vals_thresA)
            if val_thresA < 0 : val_thresA = 0
            #print("valeur seuil G0= "+str(val_thresA))
                
            vals_thresB=np.zeros([])
            subjects_BB=subjects_B[:]
            for j, subject in enumerate(subjects_B):
                if np.all(values[subject][b_idx]>0) :
                    vals_thresB=np.append(vals_thresB,np.mean(values[subject][b_idx]))
            vals_thresB=vals_thresB[1:]
            vals_thresB = vals_thresB[~ np.isnan(vals_thresB)]
            val_thresB=np.mean(vals_thresB)-2*np.std(vals_thresB)
            if val_thresB < 0 : val_thresB = 0
            #print("valeur seuil G1= "+str(val_thresB))
            
            vals_thresC=np.zeros([])
            subjects_CC=subjects_C[:]
            for j, subject in enumerate(subjects_C):
                if np.all(values[subject][b_idx]>0) :                
                    vals_thresC=np.append(vals_thresC,np.mean(values[subject][b_idx]))
            vals_thresC=vals_thresC[1:]
            vals_thresC = vals_thresC[~ np.isnan(vals_thresC)]
            val_thresC=np.mean(vals_thresC)-2*np.std(vals_thresC)
            if val_thresC < 0 : val_thresC = 0
            #print("valeur seuil G2= "+str(val_thresC))
            
            vals_thresD=np.zeros([])
            subjects_DD=subjects_D[:]
            for j, subject in enumerate(subjects_D):  
                if np.all(values[subject][b_idx]>0) :                
                    vals_thresD=np.append(vals_thresD,np.mean(values[subject][b_idx]))
            vals_thresD=vals_thresD[1:]
            vals_thresD = vals_thresD[~ np.isnan(vals_thresD)]
            val_thresD=np.mean(vals_thresD)-2*np.std(vals_thresD)
            if val_thresD < 0 : val_thresD = 0
            #print("valeur seuil G3= "+str(val_thresD))
                         
            # Bring data into right format for seaborn
            data = {"position": [],
                    "fa": [],
                    "group": [],
                    "subject": []}
                                              
            for j, subject in enumerate(subjects_A):
                if ((np.mean(values[subject][b_idx]) > val_thresA) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G1")                        
                else :
                    subjects_AA.remove(subject)                    
                             
            for j, subject in enumerate(subjects_B):
                if ((np.mean(values[subject][b_idx]) > val_thresB) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G2")                        
                else :
                    subjects_BB.remove(subject)
                    
            for j, subject in enumerate(subjects_C):
                if ((np.mean(values[subject][b_idx]) > val_thresC) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G3")                        
                else :
                    subjects_CC.remove(subject)                    
                    
            for j, subject in enumerate(subjects_D):
                if ((np.mean(values[subject][b_idx]) > val_thresD) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G4")                        
                else :
                    subjects_DD.remove(subject)
                 
            # Plot
            ax = sns.lineplot(x="position", y="fa", data=data, ax=axes[i], hue="group")
                              # units="subject", estimator=None, lw=1)  # each subject as single line
            print(all_bundles[b_idx])
            ax.set(xlabel='position along tract', ylabel='metric')
            ax.set_title(all_bundles[b_idx])
            if analysis_type == "correlation" or hide_legend:
                ax.legend_.remove()
            elif analysis_type == "group" and i > 0:
                ax.legend_.remove()  # only show legend on first subplot

            alpha=0.05
            nperm=1000
            # Significance testing without multiple correction of bundles
            if not correct_mult_tract_comp:
                values_allp = [values[s][b_idx] for s in subjects_AA + subjects_BB + subjects_CC + subjects_DD ]  # [subjects, NR_POINTS]
                #alphaFWE, clusterFWE = get_corrected_alpha(values_allp, meta_data, analysis_type, subjects_AA, subjects_BB,alpha,all_bundles, nperm, b_idx)
                clusterFWE=10
                alphaFWE=0.05

            # Calc p-values
            pvalues = np.zeros(NR_POINTS)
            stats = np.zeros(NR_POINTS)  # for ttest: t-value, for pearson: correlation
            for jdx in range(NR_POINTS):
                if analysis_type == "group":
                    values_AA = [values[s][b_idx][jdx] for s in subjects_AA]
                    values_BB = [values[s][b_idx][jdx] for s in subjects_BB]
                    values_CC = [values[s][b_idx][jdx] for s in subjects_CC]
                    values_DD = [values[s][b_idx][jdx] for s in subjects_DD]
                    #stats[jdx], pvalues[jdx] = scipy.stats.kruskal(values_A, values_B,values_C, values_D)
                    stats[jdx], pvalues[jdx]=scipy.stats.f_oneway(values_AA, values_BB, values_CC, values_DD)
                else:
                    values_controls = [values[s][b_idx][jdx] for s in subjects_A]
                    stats[jdx], pvalues[jdx] = scipy.stats.pearsonr(values_controls, meta_data["target"].values)
            
            if np.any(pvalues<alphaFWE):
                print(all_bundles[b_idx])
                print(para)
                print(len(subjects_A)+len(subjects_B))
                print(len(subjects_A))
                print(len(subjects_B))                            
                print(len(subjects_AA)+len(subjects_BB))
                print(len(subjects_AA))
                print(len(subjects_BB))                
                    
            # Plot significant areas
            if show_detailed_p:
                ax2 = axes[i].twinx()
                ax2.bar(range(len(pvalues)), -np.log10(pvalues), color="gray", edgecolor="none", alpha=0.5)
                ax2.plot([0, NR_POINTS-1], (-np.log10(alphaFWE),)*2, color="red", linestyle=":")
                ax2.set(xlabel='position', ylabel='-log10(p)')
            else:
                sig_areas = get_significant_areas(pvalues, 1, alphaFWE)
                sig_areas = sig_areas * np.quantile(np.array(data["fa"]), 0.98)
                sig_areas[sig_areas == 0] = np.quantile(np.array(data["fa"]), 0.02)
                axes[i].plot(range(len(sig_areas)), sig_areas, color="red", linestyle=":")
                sig_areas2 = get_significant_areas(pvalues, clusterFWE, alpha)
                sig_areas2 = sig_areas2 * np.quantile(np.array(data["fa"]), 0.98)
                sig_areas2[sig_areas2 == 0] = np.quantile(np.array(data["fa"]), 0.02)
                axes[i].plot(range(len(sig_areas2)), sig_areas2, color="green", linestyle="-")
            # Plot text
            if FWE_method == "alphaFWE":
                #axes[i].annotate("alphaFWE:   {}".format(format_number(alphaFWE)),
                #                 (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                #                 fontsize=10)
                axes[i].annotate("min p-value: {}".format(format_number(pvalues.min())),
                                 (0, 0), (0, -45), xycoords='axes fraction', textcoords='offset points', va='top',
                                 fontsize=10)
            else:
                axes[i].annotate("clusterFWE:   {}".format(clusterFWE),
                                 (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                                 fontsize=10)
            STR=str(len(subjects_AA)+len(subjects_BB)+len(subjects_CC)+len(subjects_DD))+"/"+str(len(subjects_AA))+"/"+str(len(subjects_BB))+"/"+str(len(subjects_CC))+"/"+str(len(subjects_DD))
            axes[i].annotate(STR,
                (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
                fontsize=10)
    
            stats_label = "t-value:      " if analysis_type == "group" else "corr.coeff.: "
            #axes[i].annotate(stats_label + "   {}".format(format_number(stats[pvalues.argmin()])),
            #                 (0, 0), (0, -55), xycoords='axes fraction', textcoords='offset points', va='top',
            #                 fontsize=10)

        plt.tight_layout()
        plt.savefig("/mnt/d//LINUX/CogPhenoPark/"+para+"_groups_2std_ANOVA_cc.png", dpi=200)

Using 31 manually specified bundles.






  0%|          | 0/31 [00:00<?, ?it/s]



  3%|▎         | 1/31 [00:09<04:37,  9.24s/it]

AF_left
AF_left
FA
56
41
15
55
40
15






  6%|▋         | 2/31 [00:18<04:29,  9.29s/it]

AF_right
AF_right
FA
56
41
15
55
40
15


...

T_OCC_right
T_OCC_right
density
56
41
15
56
41
15






 97%|█████████▋| 30/31 [04:28<00:08,  8.86s/it]

ST_FO_left






100%|██████████| 31/31 [04:36<00:00,  8.93s/it]

ST_FO_right


In [152]:
sig_areas2 = get_significant_areas(pvalues, clusterFWE, alpha)
sig_areas2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Boxplot

In [27]:
values = {}
len(values)

0

In [13]:
for para in (['''MD''','''FA''','''RD''','''AD''','''density''']): 
    for group_vs in ([''' ''']):

        ###############
        FWE_method = "alphaFWE"
        show_detailed_p = False
        hide_legend = False
        show_color_bar = True  # colorbar on 3D plot
        nperm = 5000
        alpha=0.05
        correct_mult_tract_comp = False
        base_path, meta_data, selected_bundles, plot_3D_path = parse_subjects_file("/mnt/d//LINUX/CogPhenoPark/dataTractSeg/Tractometry_template.txt")
        analysis_type = "group"
        plot_3D_type="none"

        ###########
        #all_bundles = dataset_specific_utils.get_bundle_names("All_tractometry")[1:]
        all_bundles = dataset_specific_utils.get_bundle_names("All")[1:]
        values = {}  
        for subject in meta_data["subject_id"]:
            raw = np.loadtxt(base_path.replace("SUBJECT_ID", subject).replace("PARA",para), delimiter=";", skiprows=1).transpose()
            values[subject] = raw

        NR_POINTS = values[meta_data["subject_id"][0]].shape[1]
        #selected_bun_indices = [bundles.index(b) for b in selected_bundles]
        selected_bun_indices = [all_bundles.index(b) for b in selected_bundles]

        confound_names = list(meta_data.columns[2:])

        cols = 5
        rows = math.ceil(len(selected_bundles) / cols)

        a4_dims = (cols*3, rows*7)
        f, axes = plt.subplots(rows, cols, figsize=(a4_dims))

        axes = axes.flatten()
        sns.set(font_scale=1.2)
        sns.set_style("whitegrid")

        subjects_A = list(meta_data[meta_data["group"] == 0]["subject_id"])
        subjects_B = list(meta_data[meta_data["group"] == 1]["subject_id"])
        subjects_C = list(meta_data[meta_data["group"] == 2]["subject_id"])
        subjects_D = list(meta_data[meta_data["group"] == 3]["subject_id"])
 
        # Correct for confounds
        values = correct_for_confounds(values, meta_data, all_bundles, selected_bun_indices, NR_POINTS, analysis_type,confound_names)

        print(len(subjects_A))
        print(len(subjects_B))
        print(len(subjects_C))
        print(len(subjects_D))
        cpt=0
        for i,b_idx in enumerate(selected_bun_indices):
#            print(all_bundles[b_idx])
            ############
            vals_thresA=np.zeros([])
            subjects_AA=subjects_A[:]
            for subject in subjects_A:
                if np.all(values[subject][b_idx]>0) :
                    vals_thresA=np.append(vals_thresA,np.mean(values[subject][b_idx]))
            vals_thresA=vals_thresA[1:]
            vals_thresA = vals_thresA[~ np.isnan(vals_thresA)]
            val_thresA=np.mean(vals_thresA)-2*np.std(vals_thresA)
            if val_thresA < 0 : val_thresA = 0
            #print("valeur seuil G0= "+str(val_thresA))
                
            vals_thresB=np.zeros([])
            subjects_BB=subjects_B[:]
            for j, subject in enumerate(subjects_B):
                if np.all(values[subject][b_idx]>0) :
                    vals_thresB=np.append(vals_thresB,np.mean(values[subject][b_idx]))
            vals_thresB=vals_thresB[1:]
            vals_thresB = vals_thresB[~ np.isnan(vals_thresB)]
            val_thresB=np.mean(vals_thresB)-2*np.std(vals_thresB)
            if val_thresB < 0 : val_thresB = 0
            #print("valeur seuil G1= "+str(val_thresB))
            
            vals_thresC=np.zeros([])
            subjects_CC=subjects_C[:]
            for j, subject in enumerate(subjects_C):
                if np.all(values[subject][b_idx]>0) :                
                    vals_thresC=np.append(vals_thresC,np.mean(values[subject][b_idx]))
            vals_thresC=vals_thresC[1:]
            vals_thresC = vals_thresC[~ np.isnan(vals_thresC)]
            val_thresC=np.mean(vals_thresC)-2*np.std(vals_thresC)
            if val_thresC < 0 : val_thresC = 0
            #print("valeur seuil G2= "+str(val_thresC))
            
            vals_thresD=np.zeros([])
            subjects_DD=subjects_D[:]
            for j, subject in enumerate(subjects_D):  
                if np.all(values[subject][b_idx]>0) :                
                    vals_thresD=np.append(vals_thresD,np.mean(values[subject][b_idx]))
            vals_thresD=vals_thresD[1:]
            vals_thresD = vals_thresD[~ np.isnan(vals_thresD)]
            val_thresD=np.mean(vals_thresD)-2*np.std(vals_thresD)
            if val_thresD < 0 : val_thresD = 0
            #print("valeur seuil G3= "+str(val_thresD))
                         
            # Bring data into right format for seaborn
            data = {"position": [],
                    "fa": [],
                    "group": [],
                    "subject": []}
                                              
            for j, subject in enumerate(subjects_A):
                if ((np.mean(values[subject][b_idx]) > val_thresA) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G1")                        
                else :
                    subjects_AA.remove(subject)                    
                             
            for j, subject in enumerate(subjects_B):
                if ((np.mean(values[subject][b_idx]) > val_thresB) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G2")                        
                else :
                    subjects_BB.remove(subject)
                    
            for j, subject in enumerate(subjects_C):
                if ((np.mean(values[subject][b_idx]) > val_thresC) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G3")                        
                else :
                    subjects_CC.remove(subject)                    
                    
            for j, subject in enumerate(subjects_D):
                if ((np.mean(values[subject][b_idx]) > val_thresD) & (np.all(values[subject][b_idx]>0))) :
                    for position in range(NR_POINTS):
                        data["position"].append(position)
                        data["subject"].append(subject)
                        data["fa"].append(values[subject][b_idx][position])
                        data["group"].append("G4")                        
                else :
                    subjects_DD.remove(subject)

            values_AA=vals_thresA[vals_thresA>val_thresA]
            values_BB=vals_thresB[vals_thresB>val_thresB]
            values_CC=vals_thresC[vals_thresC>val_thresC]
            values_DD=vals_thresD[vals_thresD>val_thresD]
            stat_val,p_val=scipy.stats.f_oneway(vals_thresA[vals_thresA>val_thresA], vals_thresB[vals_thresB>val_thresB], vals_thresC[vals_thresC>val_thresC], vals_thresD[vals_thresD>val_thresD])
                
            # Plot
            if p_val < 0.05 :
                ax = sns.violinplot(x="group", y="fa", data=data,ax=axes[cpt],inner="point")
                cpt=cpt+1            
                ax.set_title(all_bundles[b_idx])
                axes[cpt].annotate("p-value: {}".format(format_number(p_val)),
                    (0, 0), (0, -25), xycoords='axes fraction', textcoords='offset points', va='top',
                    fontsize=10)
                STR=str(len(subjects_AA)+len(subjects_BB)+len(subjects_CC)+len(subjects_DD))+"/"+str(len(subjects_AA))+"/"+str(len(subjects_BB))+"/"+str(len(subjects_CC))+"/"+str(len(subjects_DD))
                axes[cpt].annotate(STR,
                    (0, 0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top',
                    fontsize=10)
            
                print(p_val)
                print(all_bundles[b_idx])
                print(para)
                print(len(subjects_AA)+len(subjects_BB)+len(subjects_CC)+len(subjects_DD))
                print(len(subjects_AA))
                print(len(subjects_BB))
                print(len(subjects_CC))
                print(len(subjects_DD))
                STR="two_samples_ttest : "
                stat, pvalue = scipy.stats.ttest_ind(values_AA, values_BB)
                if (pvalue<0.05):
                    STR=STR+" G1 Vs G2 : "+ str(format_number(pvalue))
                stat, pvalue = scipy.stats.ttest_ind(values_AA, values_CC)
                if (pvalue<0.05):
                    STR=STR+" G1 Vs G3 : "+ str(format_number(pvalue))
                stat, pvalue = scipy.stats.ttest_ind(values_AA, values_DD)
                if (pvalue<0.05):
                    STR=STR+" G1 Vs G4 : "+ str(format_number(pvalue))               
                stat, pvalue = scipy.stats.ttest_ind(values_BB, values_CC)
                if (pvalue<0.05):
                    STR=STR+" G2 Vs G3 : "+ str(format_number(pvalue))
                stat, pvalue = scipy.stats.ttest_ind(values_BB, values_DD)
                if (pvalue<0.05):
                    STR=STR+" G2 Vs G4 : "+ str(format_number(pvalue))
                stat, pvalue = scipy.stats.ttest_ind(values_CC, values_DD)
                if (pvalue<0.05):
                    STR=STR+" G3 Vs G4 : "+ str(format_number(pvalue))
                axes[cpt].annotate(STR,
                    (0, 0), (0, -45), xycoords='axes fraction', textcoords='offset points', va='top',
                    fontsize=10)
                print(STR)

            
    plt.tight_layout()
    plt.show()
    plt.savefig("/mnt/d//LINUX/CogPhenoPark/"+para+"_violinplot_2std_72.png", dpi=200) 
    plt.close()

Using 31 manually specified bundles.
39
14
22
29
0.0009466167495816722
CST_right
MD
94
36
14
17
27
two_samples_ttest :  G1 Vs G3 : 0.0145 G1 Vs G4 : 0.0002 G2 Vs G4 : 0.0095
0.028372013255384167
FX_left
MD
88
32
12
19
25
two_samples_ttest :  G3 Vs G4 : 0.0144
0.008191165791659101
MCP
MD
99
38
13
20
28
two_samples_ttest :  G1 Vs G4 : 0.0021 G3 Vs G4 : 0.0083


<ipython-input-13-dd64b90c3b14>:198: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  plt.tight_layout()


Using 31 manually specified bundles.


IndexError: index 50 is out of bounds for axis 0 with size 50

In [99]:
print(len(subjects_A))
print(len(vals_thresA))
print(len(subjects_B))
print(len(vals_thresB))
print(len(subjects_C))
print(len(vals_thresC))
print(len(subjects_D))
print(len(vals_thresD))
meta_data

36
41
15
15
23
21
30
27


,subject_id,group,center,sex,age,education
0,640524MB240413,3,1,0,48.777550,10
1,160858ZFK130321,3,1,0,54.592745,12
2,480227MS04042013,3,1,0,65.059548,8
3,470214CP110413,3,1,1,66.154689,10
4,470130MY220513,3,1,1,66.329911,10
...,...,...,...,...,...,...
104,T84072,0,2,1,47.997262,10
105,T84074,0,2,1,62.201232,12
106,T84075,0,2,1,70.354552,18
107,T84076,0,2,1,65.267625,15


In [65]:
vals_thresD[vals_thresD>val_thresD]
print(subjects_B)
print(vals_thresB)
print(type(subjects_B))
print(type(vals_thresB))
print(len(subjects_B))
print(len(vals_thresB))
for j, subject in enumerate(subjects_B):  
    if np.all(values[subject][b_idx]>0) :                
        print(subject)
        print(np.mean(values[subject][b_idx]))
        #vals_thresD=np.append(vals_thresD,)

['480412JL141113', '581109PF281113', '710624PD121213', '381103AD070414', '181037MB070714', 'T84006', 'T84027', 'T84034', 'T84041', 'T84054', 'T84055', 'T84059', 'T84067', 'T84077', 'T84078']
[ 3500.02541986 15656.082298    8066.99031576  4651.04558263
 14537.03033279  8953.98051638 13041.53036631 12726.01288813
 10566.87128861 10027.35028094 12862.01528346 10424.00050256
  9039.37985781  6895.39748009  9932.07934947  4187.05467031]
<class 'list'>
<class 'numpy.ndarray'>
15
16
480412JL141113
15656.08229800108
581109PF281113
8066.990315755977
710624PD121213
4651.045582630457
381103AD070414
14537.03033278724
181037MB070714
8953.980516381256
T84006
13041.530366313342
T84027
12726.012888127854
T84034
10566.871288610686
T84041
10027.35028094288
T84054
12862.015283464021
T84055
10424.000502563984
T84059
9039.379857806462
T84067
6895.397480093656
T84077
9932.079349472217
T84078
4187.054670305879


In [41]:
plot_tractometry_with_pvalue(values, meta_data, all_bundles,selected_bundles,"/NAS/dumbo/protocoles/CogPhenoPark/",
                             0.05, FWE_method, analysis_type, correct_mult_tract_comp,
                             show_detailed_p, nperm=5000, hide_legend=False,
                             plot_3D_path=plot_3D_path, plot_3D_type="pval",
                             tracking_format="tck", tracking_dir="auto",
                             show_color_bar=show_color_bar)

<ipython-input-41-e2a0c68a2347>:1: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  plt.tight_layout()


In [8]:
def t_stat(y, X, c):
    """ betas, t statistic and significance test given data, design matrix, contrast

    This is OLS estimation; we assume the errors to have independent
    and identical normal distributions around zero for each $i$ in
    $\e_i$ (i.i.d).
    """
    # Make sure y, X, c are all arrays
    y = np.asarray(y)
    X = np.asarray(X)
    c = np.atleast_2d(c).T  # As column vector
    # Calculate the parameters - b hat
    beta = npl.pinv(X).dot(y)
    # The fitted values - y hat
    fitted = X.dot(beta)
    # Residual error
    errors = y - fitted
    # Residual sum of squares
    RSS = (errors**2).sum(axis=0)
    # Degrees of freedom is the number of observations n minus the number
    # of independent regressors we have used.  If all the regressor
    # columns in X are independent then the (matrix rank of X) == p
    # (where p the number of columns in X). If there is one column that
    # can be expressed as a linear sum of the other columns then
    # (matrix rank of X) will be p - 1 - and so on.
    df = X.shape[0] - npl.matrix_rank(X)
    # Mean residual sum of squares
    MRSS = RSS / df
    # calculate bottom half of t statistic
    SE = np.sqrt(MRSS * c.T.dot(npl.pinv(X.T.dot(X)).dot(c)))
    t = c.T.dot(beta) / SE
    # Get p value for t value using cumulative density dunction
    # (CDF) of t distribution
    ltp = t_dist.cdf(t, df) # lower tail p
    p = 1 - ltp # upper tail p
    return beta, t, df, p

In [37]:
from openpyxl import load_workbook
import glob as glob
from shutil import copyfile
import pandas as pd 

file_names=glob.glob('/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_csvTractSeg_72/*_L2_tractometry_72.csv')
for file_nameL2 in file_names :
    file_nameL3 = file_nameL2.replace("_L2_tractometry_72.csv","_L3_tractometry_72.csv")     
    print(file_nameL2)    
    df_L2 = pd.read_csv(file_nameL2,header=0,index_col=False,sep = ';')
    df_L3 = pd.read_csv(file_nameL3,header=0,index_col=False,sep = ';')
    
    file_nameRD = file_nameL2.replace("_L2_tractometry_72.csv","_RD_tractometry_72.csv")     
    RD=(df_L2.values+df_L3.values)/2
    df = pd.DataFrame(RD)
    print(df_L2.columns)
    df.to_csv(file_nameRD,sep = ';',header=list(df_L2.columns),index=False)
    
    #name1=file_name.replace("/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/","").replace(".csv","")
    #name2=file_nameR.replace("/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/","").replace(".csv","")
    #wb1 = load_workbook(filename = file_name)
    #wb2 = load_workbook(filename = file_nameR)
    #sheet_ranges1 = wb1[name1]
    #sheet_ranges2 = wb1[name2]
    #print(sheet_ranges1["A2"].value)
    #print(sheet_ranges2["A2"].value)
    
    #copyfile(file_name,file_nameR)     

/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_csvTractSeg_72/100269SD100714_L2_tractometry_72.csv
Index(['AF_left', 'AF_right', 'ATR_left', 'ATR_right', 'CA', 'CC_1', 'CC_2',
       'CC_3', 'CC_4', 'CC_5', 'CC_6', 'CC_7', 'CG_left', 'CG_right',
       'CST_left', 'CST_right', 'MLF_left', 'MLF_right', 'FPT_left',
       'FPT_right', 'FX_left', 'FX_right', 'ICP_left', 'ICP_right', 'IFO_left',
       'IFO_right', 'ILF_left', 'ILF_right', 'MCP', 'OR_left', 'OR_right',
       'POPT_left', 'POPT_right', 'SCP_left', 'SCP_right', 'SLF_I_left',
       'SLF_I_right', 'SLF_II_left', 'SLF_II_right', 'SLF_III_left',
       'SLF_III_right', 'STR_left', 'STR_right', 'UF_left', 'UF_right', 'CC',
       'T_PREF_left', 'T_PREF_right', 'T_PREM_left', 'T_PREM_right',
       'T_PREC_left', 'T_PREC_right', 'T_POSTC_left', 'T_POSTC_right',
       'T_PAR_left', 'T_PAR_right', 'T_OCC_left', 'T_OCC_right', 'ST_FO_left',
       'ST_FO_right', 'ST_PREF_left', 'ST_PREF_right', 'ST_PREM_left',
       'ST_PREM

FileNotFoundError: [Errno 2] File /mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_csvTractSeg_72/340910RL270314_L3_tractometry_72.csv does not exist: '/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_csvTractSeg_72/340910RL270314_L3_tractometry_72.csv'

In [38]:
from openpyxl import load_workbook
import glob as glob
from shutil import copyfile
import pandas as pd 

file_names=glob.glob('/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/*.csv')
for file_nameL2 in file_names :
    df_L2 = pd.read_csv(file_nameL2,index_col = 0,sep = ';')
    if (np.any(df_L2.values<0)) :
        print(file_nameL2)

/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_400726PS020614.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_411003DG200214.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_450723AS270114_PB.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_480604JH160114_PB.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_490815MPD280313.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_530802JV130114_PB.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/L3__sub_T84039.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_100269SD100714.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_160858ZFK130321.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_181037MB070714.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_340531NS060613.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_340910RL270314.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_360108LW050214.csv
/mnt/d/LI

/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84067.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84072.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84074.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84075.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84076.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84077.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84078.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/MO__sub_T84079.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/RD__sub_400726PS020614.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/RD__sub_530802JV130114_PB.csv
/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/RD__sub_T84039.csv


In [20]:
import pandas as pd 
df_L2 = pd.read_csv("/mnt/d/LINUX/CogPhenoPark/dataTractSeg/nbTrack_t.csv",index_col = 0,sep = ';')

In [23]:
boxplot = df_L2.boxplot(column=["AF_left"])
plt.show()
df_L2.describe()

<ipython-input-23-78024de4cd58>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


,AF_left,AF_right,ATR_left,ATR_right,CA,CC_1,CC_2,CC_3,CC_4,CC_5,...,T_POSTC_right,T_PREC_left,T_PREC_right,T_PREF_left,T_PREF_right,T_PREM_left,T_PREM_right,UF_left,UF_right,whole_brain
count,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,128.000000,1.290000e+02
mean,22738.077519,10080.348837,34603.201550,26405.178295,668.015504,42180.620155,87823.720930,5059.821705,10200.612403,143696.658915,...,38229.806202,112944.968992,92681.472868,67518.875969,60770.488372,34708.209302,17653.798450,17624.279070,30543.453125,4.486058e+06
std,10121.382322,6720.831027,14965.974523,12907.336666,1355.694235,29408.762072,24045.273526,3869.573377,9309.671410,47328.498712,...,26402.414449,48909.362325,48416.678381,18188.042320,18265.961757,21049.121909,14382.459667,10950.492474,14901.626444,1.197462e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.030000e+02
25%,16226.000000,5529.000000,25096.000000,17439.000000,0.000000,17979.000000,75704.000000,1962.000000,1435.000000,114419.000000,...,14873.000000,85076.000000,56738.000000,56773.000000,51730.000000,17981.000000,6103.000000,10569.000000,22576.000000,3.885725e+06
50%,21018.000000,8498.000000,35874.000000,27162.000000,55.000000,43330.000000,88979.000000,4774.000000,8789.000000,147067.000000,...,36683.000000,124117.000000,95020.000000,68454.000000,61407.000000,32563.000000,14903.000000,17258.000000,31089.000000,4.613071e+06
75%,28295.000000,13620.000000,43958.000000,34889.000000,556.000000,63283.000000,105333.000000,7774.000000,17808.000000,178113.000000,...,56457.000000,145578.000000,126980.000000,77833.000000,72007.000000,48449.000000,25344.000000,22524.000000,40295.750000,5.282473e+06
max,66331.000000,38646.000000,74123.000000,79710.000000,7395.000000,150232.000000,142774.000000,16781.000000,35281.000000,255515.000000,...,97175.000000,205857.000000,208622.000000,111866.000000,127588.000000,84520.000000,58622.000000,55281.000000,63152.000000,6.590202e+06


In [16]:
fig, axes = plt.subplots()
sns.violinplot('AF_left',df_L2)
plt.show()

ValueError: Could not interpret input 'AF_left'

In [12]:
file_names=glob.glob('/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_tracto_MNI/w_AF_left.*.nii.gz')
cpt=0
restot=np.zeros((122,182, 218, 182))
for i,file_name in enumerate(file_names) :
    img = nib.load(file_name)
    restot[i,:,:,:] = img.get_fdata()

test=np.mean(restot,axis=0)

In [15]:
img = nib.Nifti1Image(test, np.eye(4))

img.get_data_dtype() == np.dtype(np.int16)
nib.save(img, '/mnt/d/LINUX/CogPhenoPark/dataTractSeg/w_AF_left_mean.nii.gz')

In [4]:
def mutual_information(hgram):
    """ Mutual information for joint histogram
    """
    # Convert bins counts to probability values
    pxy = hgram / float(np.sum(hgram))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :] # Broadcast to multiply marginals
    # Now we can do the calculation using the pxy, px_py 2D arrays
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

In [20]:
time()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


()

In [ ]:
from scipy.spatial import distance
from sklearn.metrics.cluster import normalized_mutual_info_score
import pandas as pd 

df_full = pd.read_csv("/mnt/d/LINUX/CogPhenoPark//dataTractSeg/nbTrack_t.csv",index_col = 0,sep = ';')
df_tractometry=df_L2 = pd.read_csv("/mnt/d/LINUX/CogPhenoPark/dataTractSeg/ind_stats/AD__sub_100269SD100714.csv",sep = ';')
resSimi=np.zeros((129,129,72))
for z,trk in enumerate(df_full.columns[0:-1]):
    print(trk)
    file_names=glob.glob('/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_tracto_MNI/w_'+trk+'.*.nii.gz')
    file_names2=glob.glob('/mnt/d/LINUX/CogPhenoPark/dataTractSeg/CogPhenoPark_tracto_MNI/w_'+trk+'.*.nii.gz')
    for i,file_name in enumerate(file_names) :
        img = nib.load(file_name)
        data= img.get_fdata()
        print(i)
        for j,file_name2 in enumerate(file_names2) :
            if ( resSimi[j,i,z] == 0 ) :
                img2 = nib.load(file_name2)
                data2= img2.get_fdata()
                #hist_2d, x_edges, y_edges = np.histogram2d(data.ravel(),data2.ravel(),bins=200)
                tmp_val=normalized_mutual_info_score(data.ravel(),data2.ravel())#mutual_information(hist_2d) 
                #print(file_name)
                print(j)
                #print(file_name2)
                #print(tmp_val)
                resSimi[i,j,z]=tmp_val
            else :
                resSimi[i,j,z] == resSimi[j,i,z] 

AF_left
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
2
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
24
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
25
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
26
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
8

119
120
121
53
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
54
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
55
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
56
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
57
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


113
114
115
116
117
118
119
120
121
107
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
108
108
109
110
111
112
113
114
115
116
117
118
119
120
121
109
109
110
111
112
113
114
115
116
117
118
119
120
121
110
110
111
112
113
114
115
116
117
118
119
120
121
111
111
112
113
114
115
116
117
118
119
120
121
112
112
113
114
115
116
117
118
119
120
121
113
113
114
115
116
117
118
119
120
121
114
114
115
116
117
118
119
120
121
115
115
116
117
118
119
120
121
116
116
117
118
119
120
121
117
117
118
119
120
121
118
118
119
120
121
119
119
120
121
120
120
121
121
121
AF_right
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14


51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
22
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
23
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
24
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
6

120
121
50
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
51
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
52
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
53
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
54
54
55
56
57
58
59
60
61
62
63
64
65
66
67
6

98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
99
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
100
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
101
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
102
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
103
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
104
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
105
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
106
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
107
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
108
108
109
110
111
112
113
114
115
116
117
118
119
120
121
109
109
110
111
112
113
114
115
116
117
118
119
120
121
110
110
111
112
113
114
115
116
117
118
119
120
121
111
111
112
113
114


33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
20
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
21
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
22
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
3

53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
48
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
49
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
50
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
